In [1]:
import numpy as np
import pymongo
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import mongo as MongoClient
import sys
print(sys.executable)


c:\Users\Utilisateur\anaconda3\python.exe


In [2]:


uri = "mongodb://localhost:27017"

client = MongoClient(uri)

db = client["movie"]
collection_movie = db["movie_json"]
collection_user = db["user_json"]


TypeError: 'module' object is not callable

In [ ]:
# essais interrogation data movie Q0
documents = collection_movie.find().limit(1)
for document in documents:
    print("Q0",document)

# essais interrogation data user Q0
documents = collection_user.find().limit(1)
for document in documents:
    print("Q0", document)
print(documents)


NameError: name 'collection_movie' is not defined

In [ ]:
# combien de categorie de film y a til 
# Définir la requête d'agrégation
pipeline = [
    {
        '$project': {
            'genres': {'$split': ['$genres', '|']}  # Séparer les genres en une liste
        }
    },
    {
        '$unwind': '$genres'  # Décomposer les listes de genres en documents individuels
    },
    {
        '$group': {
            '_id': '$genres',  # Grouper par genre
            'count': {'$sum': 1}  # Compter le nombre de films dans chaque genre
        }
    },
    {
        '$project': {
            '_id': 0,  # Ne pas inclure l'identifiant
            'genre': '$_id',  # Renommer l'attribut _id en genre
            'count': 1  # Inclure le nombre de films dans chaque genre
        }
    }
]

# Exécuter la requête d'agrégation
result = collection_movie.aggregate(pipeline)

# Afficher les résultats
for genre in result:
    print(genre['genre'], "-", genre['count'])



In [ ]:
distinct_genres = collection_movie.aggregate([
    {
        "$project": {
            "genres": {"$split": ["$genres", "|"]}
        }
    },{
        "$unwind": "$genres"
    },
    {
        "$group": {
            "_id": "$genres",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
])

[genre for genre in distinct_genres]


In [ ]:
# Voici le filtre pour avoir acces uniquement à l'action et a la comedie 
movie_ids = { 'genres': { '$in': ["Children's", "Horror"] } }
result = collection_movie.find(movie_ids)
for film in result:
    print(film['title'], "-", film['genres'])

In [ ]:
# Définir la requête pour la jointure entre les collections
pipeline = [
    {
        '$match': movie_ids  # Filtre les films en fonction des genres spécifiés
    },
    {
        '$lookup': {
            'from': 'user_json',  # Nom de la collection des utilisateurs
            'localField': '_id',  # Champ local à partir duquel faire la jointure (dans la collection de films)
            'foreignField': 'movie_id',  # Champ étranger à partir duquel faire la jointure (dans la collection d'utilisateurs)
            'as': 'user_ratings'  # Nom de la liste dans laquelle stocker les évaluations des utilisateurs
        }
    },
    {
        '$unwind': {
            'path': '$user_ratings',  # Décompose les évaluations des utilisateurs en documents individuels
            'preserveNullAndEmptyArrays': True  # Préserver les documents sans correspondance lors de la jointure
        }
    },
    {
        '$project': {
            'title': 1,  # Inclure le titre du film
            'genres': 1,  # Inclure les genres du film
            'user_id': '$user_ratings.user_id',  # Récupérer l'identifiant de l'utilisateur
            'rating': '$user_ratings.rating'  # Récupérer l'évaluation de l'utilisateur
        }
    }
]

# Exécuter la requête d'agrégation
result = collection_movie.aggregate(pipeline)

# Afficher les résultats
for film in result:
    print("Titre:", film['title'])
    print("Genres:", film['genres'])
    print("Identifiant de l'utilisateur:", film['user_id'])
    print("Évaluation:", film['rating'])
    print("--------------------------------")

